In [1]:
import pandas as pd
import numpy as np
import random as rnd


import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.model_selection
import sklearn.svm as svm
import sklearn.metrics
from matplotlib import pyplot

In [3]:
#Importation des données
titanic_train='/Users/Naoufali/Desktop/M2 Ekap/titanic/train.csv'
titanic_test='/Users/Naoufali/Desktop/M2 Ekap/titanic/test.csv'
gender='/Users/Naoufali/Desktop/M2 Ekap/titanic/gender_submission.csv'

train = pd.read_csv(titanic_train)
test = pd.read_csv(titanic_test)
gender_submission = pd.read_csv(gender)

In [ ]:
#Visualisation
train.head()

In [ ]:
#Visualisation
test.head()

In [ ]:
#Visualisation

gender_submission.head()

In [6]:
train['Cabin?'] = np.where(pd.isnull(train['Cabin']), 0,1)
predictors = ['Pclass','Sex', 'Age', 'SibSp','Parch', 'Fare', 'Cabin?', 'Embarked']

In [7]:
#Suppression des valeurs Nan + maj 
def impute_age(cols):
    age = cols[0]
    sex = cols[1]
    pclass = cols[2]
    if pd.isnull(age):
        if sex == 'female':
            if pclass == 1:
                return 35
            elif pclass == 2:
                return 28
            elif pclass == 3:
                return 21.5
            else:
                return np.nan
        elif sex == 'male':
            if pclass == 1:
                return 40
            elif pclass == 2:
                return 30
            elif pclass == 3:
                return 25
            else:
                return np.nan
    else:
        return age

    
train['Age']=train[['Age','Sex','Pclass']].apply(impute_age,axis=1)

In [8]:
male = pd.get_dummies(train['Sex'], drop_first=True)
port = pd.get_dummies(train['Embarked'], drop_first=True)
train = pd.concat([train, male, port],axis=1)
train[pd.isnull(train['Embarked'])==True]
train.loc[61,'Q']=0.33
train.loc[61,'S']=0.33
train.loc[829,'Q']=0.33
train.loc[829,'S']=0.33
predictors_num = ['Pclass', 'male', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin?', 'Q','S']

In [9]:
yX_tr, X_test, y_tr, y_test = sklearn.model_selection.train_test_split(train[predictors_num],train['Survived'], random_state = 1)

In [10]:
X_tr

,Pclass,male,Age,SibSp,Parch,Fare,Cabin?,Q,S
35,1,1,42.0,1,0,52.0000,0,0.0,1.0
46,3,1,25.0,1,0,15.5000,0,1.0,0.0
453,1,1,49.0,1,0,89.1042,1,0.0,0.0
291,1,0,19.0,1,0,91.0792,1,0.0,0.0
748,1,1,19.0,1,0,53.1000,1,0.0,1.0
...,...,...,...,...,...,...,...,...,...
715,3,1,19.0,0,0,7.6500,1,0.0,1.0
767,3,0,30.5,0,0,7.7500,0,1.0,0.0
72,2,1,21.0,0,0,73.5000,0,0.0,1.0
235,3,0,21.5,0,0,7.5500,0,0.0,1.0


In [25]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split 
#On importe la fonction train_test_split de la librairie sklearn (fonction pour diviser aléatoirement une base)
from sklearn import metrics #la librairie metrics permet de faire des calcul sur les modèles de sklearn
from sklearn.model_selection import cross_val_score


MLP_model = MLPClassifier()
MLP_model.fit(X_tr,y_tr)
y_pred=MLP_model.predict(X_test)

/Users/Naoufali/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [26]:
from sklearn.neural_network import MLPClassifier

MLP_model = MLPClassifier()
MLP_model.fit(X_tr,y_tr)


#Taux de prédiction sur l'échantillon d'entrainement(in-sample), ici toute la base : 100%
print("Taux de prédiction sur échantillon d'entrainement : {}" .format(MLP_model.score(X_tr,y_tr)))

#Taux de prédiction hors échantillon découpé aléatoirement:
MLP_model.fit(X_tr,y_tr)
y_pred=MLP_model.predict(X_test)
#Taux de prédiction de 95,6% 

print("Taux de prédiction sur échantillon découpé aléatoirement:", metrics.accuracy_score(y_true=y_test, y_pred=y_pred), "\n") #Taux de bonne prédiction
print(metrics.confusion_matrix(y_true=y_test, y_pred=y_pred)) #Matrice de confusion

#Taux de prédiction moyen à l'aide de la cross validation:
cv_results = cross_val_score(MLP_model, X_tr, y_tr, cv=10)
moyenne_cv_results = cv_results.mean()
print("Taux dde prédiction trouvés : {}" .format(cv_results))
print("Taux de prédiction de Cross-Validation (moyenne) : {}" .format(moyenne_cv_results))

#Le taux moyen de bonne prediction est de 95,3% sur les 10 test de modèles,
#cette valeur est moins bonne que pour les modèles SVM

/Users/Naoufali/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Taux de prédiction sur échantillon d'entrainement : 0.8218562874251497
Taux de prédiction sur échantillon découpé aléatoirement: 0.7892376681614349 

[[112  16]
 [ 31  64]]


/Users/Naoufali/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Taux dde prédiction trouvés : [0.79411765 0.70149254 0.6119403  0.67164179 0.76119403 0.74626866
 0.76119403 0.6969697  0.66666667 0.78787879]
Taux de prédiction de Cross-Validation (moyenne) : 0.7199364141857558
